In [1]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib
%matplotlib widget
matplotlib.rcParams['figure.autolayout']=True 
import numpy as np
from functools import partial
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Button, Layout, FloatSlider, Dropdown, Output
from ipywidgets import TwoByTwoLayout
from IPython.display import display, clear_output
from ase import Atoms
from sklearn.neighbors import KernelDensity

from ase.lattice.cubic import FaceCenteredCubic
from ase.md.langevin import Langevin
from ase.io.trajectory import Trajectory
from ase import units

import nglview as nv

import os

In [2]:
def compute_positions(ka,t):
    positions=0.5*np.cos(-w*t+np.arange(-10,11)*ka)+np.arange(-10,11)*2
    return positions

In [3]:
slider_ka=FloatSlider(value=0,min=-np.pi,max=np.pi,continuous_update=False,step=0.01)
def ka_change(event):
    global traj, view
    traj=Trajectory('traj_test.traj','w') #Erase data
    traj=Trajectory('traj_test.traj','a')
    ka=slider_ka.value
    for frames in np.linspace(0,100,100):
        t=2*np.pi/100/w*frames
        # t=frames
        positions=compute_positions(ka,t)
        test=[[x, 0, 0] for x in positions]
        atoms=Atoms(21*'C', positions=test)
        traj.write(atoms)

    comp_ids=[]
    for comp_id in view._ngl_component_ids:
        comp_ids.append(comp_id)

    traj_ = Trajectory("traj_test.traj")

    view.add_trajectory(traj_)

    for comp_id in comp_ids:
        view.remove_component(comp_id)

    view.clear_representations()
    view.add_representation('spacefill',source='#C',radius=0.5)
    view.control.zoom(0.6)
    view.center()
    
slider_ka.observe(ka_change)

In [4]:
ka=np.pi
traj=Trajectory('traj_test.traj','w') #Erase data
traj=Trajectory('traj_test.traj','a')
w=np.sqrt(4*1/1)*np.sin(np.abs(ka))
for frames in np.linspace(0,100,100):
    t=2*np.pi/100/w*frames
    # t=frames
    positions=compute_positions(ka,t)
    test=[[x, 0, 0] for x in positions]
    atoms=Atoms(21*'C', positions=test)
    traj.write(atoms)

traj_=Trajectory('traj_test.traj')
view=nv.show_asetraj(traj_)
view.clear_representations()
view.add_representation('spacefill',source='#C',radius=0.5)
view.player.parameters=dict(delay=20)
view.control.zoom(0.6)
view._remote_call("setSize", target="Widget", args=["500px", "300px"])

In [5]:
def update_ngl():
    global traj, view, w, ka
    traj=Trajectory('traj_test.traj','w') #Erase data
    traj=Trajectory('traj_test.traj','a')
    w=np.sqrt(4*1/1)*np.sin(np.abs(ka)/2)
    for frames in np.linspace(0,100,100):
        t=2*np.pi/100/w*frames
        # t=frames
        positions=compute_positions(ka,t)
        test=[[x, 0, 0] for x in positions]
        atoms=Atoms(21*'C', positions=test)
        traj.write(atoms)

    comp_ids=[]
    for comp_id in view._ngl_component_ids:
        comp_ids.append(comp_id)

    traj_ = Trajectory("traj_test.traj")

    view.add_trajectory(traj_)

    for comp_id in comp_ids:
        view.remove_component(comp_id)

    view.clear_representations()
    view.add_representation('spacefill',source='#C',radius=0.5)
    view.center()
    view.control.zoom(0.6)
    
    view._remote_call("setSize", target="Widget", args=["450px", "300px"])

In [7]:

def onclick(event):
    global point, ka
    #Plot point on graph
    ka=event.xdata
    w=np.sqrt(4*1/1)*np.sin(np.abs(ka)/2)
    point.set_data(ka,w)
    update_ngl()
    

In [8]:
plt.ioff()
px = 1/plt.rcParams['figure.dpi']
fig, ax = plt.subplots(figsize=(500*px,300*px))

fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False

ka_=np.linspace(-np.pi,np.pi,101)
w_=np.sqrt(4*1/1)*np.sin(np.abs(ka_)/2)
ax.plot(ka_,w_,c='dodgerblue')
point,=ax.plot([],[],'.',c='crimson',markersize=15)
fig.canvas.mpl_connect('button_press_event', onclick);
plt.ion();

In [9]:
# VBox([fig.canvas,view],layout=Layout(align_items="center"))

In [10]:
plt.ioff()
fig2, ax2 = plt.subplots(figsize=(500*px,100*px))

fig2.canvas.toolbar_visible = False
fig2.canvas.header_visible = False
fig2.canvas.footer_visible = False
fig3, ax3 = plt.subplots(figsize=(500*px,100*px))
ax2.axis('off')
fig3.canvas.toolbar_visible = False
fig3.canvas.header_visible = False
fig3.canvas.footer_visible = False
ax3.axis('off')
plt.ion()

In [13]:
TwoByTwoLayout(top_left=fig.canvas,
               top_right=view,
               bottom_left=fig2.canvas,
               bottom_right=fig3.canvas)

TwoByTwoLayout(children=(Canvas(footer_visible=False, header_visible=False, layout=Layout(grid_area='top-left'…

In [16]:
view.download_image('atom_screenshot.png')